In [241]:
import numpy as np
import pandas as pd
import requests
import json
import os
import cPickle as pickle
from time import sleep

In [49]:
api_key = ''

steam_id_url = 'http://api.steampowered.com/ISteamApps/GetAppList/v2'
steam_details_api = 'http://store.steampowered.com/api/appdetails/?appids='
ratings_api = 'http://steamspy.com/api.php?request=appdetails&appid='
app_path = '/Users/nmouterde/Documents/projectsPython/steam-game-engine/'

In [55]:
steam_ids = requests.get(steam_id_url)

In [56]:
steam_ids = json.loads(steam_ids.text)

In [57]:
df_steam_ids = pd.DataFrame.from_dict(steam_ids['applist']['apps'])

In [ ]:
games = []
i=0

In [325]:
for id_app in df_steam_ids.appid.values:
    api_call = requests.get(steam_details_api + str(id_app))
    rates_call = requests.get(ratings_api + str(id_app))
    rates = json.loads(rates_call.text)
    app_desc = json.loads(api_call.text)
    while(app_desc == None):
        sleep(120)
        api_call = requests.get(steam_details_api + str(id_app))
        rates_call = requests.get(ratings_api + str(id_app))
        rates = json.loads(rates_call.text)
        app_desc = json.loads(api_call.text)
    game = app_desc[app_desc.keys()[0]]
    if game['success']:
        game_info = game['data']
        if game_info['type'] == "game":
            #Treating special categories
            if 'categories' in game_info:
                cats = [cat['description'] for cat in game_info['categories']]
                cats = ' '.join(cats)
            else:
                cats = ' '

            if 'recommendations' not in game_info:
                rec = 0
            else:
                rec = game_info['recommendations']['total']

            if 'developers' in game_info:
                dev = game_info['developers'][0]
            else:
                dev = ' '

            if 'genres' in game_info:
                genres = [genre['description'] for genre in game_info['genres']]
                genres = ' '.join(genres)
            else:
                genre = ' '

            new_game = [game_info['steam_appid'], game_info['name'],
                          game_info['detailed_description'],
                          game_info['short_description'], 
                          game_info['about_the_game'],
                          dev,
                          game_info['publishers'][0],
                          cats, genres,
                          rates['score_rank'],
                          rec]


            games.append(new_game)
            


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500


In [327]:
df_cols = ['id', 'name',
           'f_desc', 's_desc','about',
           'dev' , 'pub', 'cat',
           'genres', 'score', 'nb_rec']

df_info = pd.DataFrame.from_records(games, columns=df_cols)       

In [332]:
df_info = df_info.drop_duplicates()

In [334]:
pickle.dump(df_info, open(os.path.join(app_path, 'data_processed/scrap_df.f'), 'wb'))

In [213]:
api_call.text
steam_details_api + str(id_app)

'http://store.steampowered.com/api/appdetails/?appids=290380'